[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/fonnesbeck/bayes_course_june_2024/blob/master/notebooks/Section2_3-Generalized_Linear_Models.ipynb)

# Bayesian GLM

In the previous section, the final example involved a Poisson regression model because the outcome was a count variable. In this section, we will expand on this type of application using Generalized Linear Models (GLMs), which are a generalization of linear regression that allows for non-normal distributions of the outcome variable.

We'll also show you a common circumstance where the assumptions built into these distributions fall apart in real life, and how to address the problem.

In [ ]:
import arviz as az
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pymc as pm
import seaborn as sns
import xarray as xr
from matplotlib.lines import Line2D

RANDOM_SEED = 8927
np.random.seed(RANDOM_SEED)
az.style.use("arviz-darkgrid")

## Poisson regression: Unbounded count data

This model is inspired by [a project by Ian Osvald](http://ianozsvald.com/2016/05/07/statistically-solving-sneezes-and-sniffles-a-work-in-progress-report-at-pydatalondon-2016/), which is concerned with understanding the various effects of external environmental factors upon the allergic sneezing of a test subject.

We're going to work with simpler data than the original study, which will allow you to clearly see the modeling stakes.

In [ ]:
sneezes = pd.read_csv("../data/poisson_sneeze.csv")
sneezes

+ The subject sneezes N times per day, recorded as `nsneeze (int)`. The data are aggregated per day, to yield a total count of sneezes on that day.
+ The subject may or may not drink alcohol during that day, recorded as `alcohol (boolean)`
+ The subject may or may not take an antihistamine medication during that day, recorded as `meds (boolean)`

We assume that sneezing occurs at some baseline rate, which increases if an antihistamine is not taken, and further increases if alcohol is consumed.

### Visualize the data and set up the model

In [ ]:
g = sns.catplot(
    x="nsneeze", col="meds", row="alcohol", data=sneezes, kind="count", color="blue"
)

# Make x-axis ticklabels less crowded
ax = g.axes[1, 0]
labels = range(len(ax.get_xticklabels(which="both")))
ax.set_xticks(labels[::5])
ax.set_xticklabels(labels[::5]);


The usual way of thinking about data coming from a Poisson distribution is as number of occurences of an event in a *given timeframe*. Here *number of sneezes per day*. The intensity parameter $\lambda$ specifies how many occurences we expect.

A nice property of the Poisson is that it's defined with only one parameter $\lambda$, which describes both the mean and variance of the Poisson, can be interpreted as the rate of events per unit -- here, if we inferred $\lambda = 2.8$, that would mean the subject is thought to sneeze about 2.8 times per day (implying in addition a $2.8$ variance).

In statistical terms, that means our likelihood is 

$$ Y_{\text{sneeze}} \sim \mathrm{Poisson}(\lambda)$$

Now, we need a prior on $\lambda$ ...

This is where the regression component comes in: remember that we want to infer the effect of meds and alcohol on the number of sneezes. So, we can use the usual linear regression formula we are familiar with: 

$$\lambda = \alpha + \beta_{\text{meds}} * \text{meds} + \beta_{\text{alcohol}} * \text{alcohol}$$

We will specify **weakly-informative priors** on the model latent variables.

$$
\begin{aligned}
\alpha &\sim \mathrm{Normal}(0, 5)\\
\beta_{\text{meds}}, \beta_{\text{alcohol}} &\sim \mathrm{Normal}(0, 1)\\
\lambda &= \alpha + \beta_{\text{meds}} * \text{meds} + \beta_{\text{alcohol}} * \text{alcohol}\\
Y_{\text{sneeze}} &\sim \mathrm{Poisson}(\lambda)\\
\end{aligned}
$$

In [ ]:
COORDS = {
    "regressor": ["meds", "alcohol"], 
    "obs_idx": sneezes.index
}

M, A, S = sneezes[["meds", "alcohol", "nsneeze"]].to_numpy().T

with pm.Model(coords=COORDS) as m_sneeze:
    # weakly informative Normal Priors
    a = pm.Normal("intercept", mu=0, sigma=5)
    b = pm.Normal("slopes", mu=0, sigma=1, dims="regressor")

    # define linear model
    mu = pm.Deterministic("mu", a + b[0] * M + b[1] * A, dims="obs_idx")

    ## Define Poisson likelihood
    y = pm.Poisson("y", mu=mu, observed=S, dims="obs_idx")

pm.model_to_graphviz(m_sneeze)


> ### Model coordinates
> 
> The model coordinates in PyMC are used to define dimensions and labels for the variables in the model. They provide a way to organize and manipulate the model's data with dimensions _names_ instead of raw _shapes_, through the `coords` and `dims` keyword arguments. In the given PyMC model, the coordinates are defined as follows:
> 
> - `regressor`: This coordinate represents the regressor variables in the model, which are `'meds'` and `'alcohol'`.
> - `obs_idx`: This coordinate represents the index of the observations in the model. It is a `RangeIndex` object with a range from 0 to 4000.
> 
> These coordinates allow for more intuitive and readable model specification, as well as easier manipulation and analysis of the model's data.


### Model Fitting

Most GLMs will be fit using the NUTS step method.

In [ ]:
with m_sneeze:
    trace_sneeze = pm.sample()

While the model sampled to completion, close inspection of the model reveals an issue that could have caused the sampler to fail. Can you spot it? In prevoious versions of PyMC, this model would have failed to sample.

Propose a more robust version below.

In [ ]:
# Write your answer here

Before looking at the results, let's take a step back: the trick we used with the exponential is actually exactly how generalized linear models are defined. The exponential here is called **a link function**, and it's used to map the **output of our linear model** (a priori allowed to live in $(-\infty, \infty)$,

### Link functions

In Generalized Linear Models (GLMs), link functions play a crucial role in connecting the linear predictor to the response variable. The concept of link functions arises from the need to model the relationship between the mean of the response variable and the linear predictor, which can take any real value.

The link function transforms the linear predictor to ensure that the predicted values of the response variable are within a valid range and satisfy the distributional assumptions of the response variable. It maps the linear predictor to the space of the relevant parameter of the response distribution.

Different types of GLMs use different link functions based on the nature of the response variable and the desired distribution. Some commonly used link functions include:

1. Identity Link: This link function is used for continuous response variables and maintains a linear relationship between the linear predictor and the mean of the response variable.

2. Logit Link: This link function is used for binary response variables and maps the linear predictor to the probability of success in a logistic regression model. It ensures that the predicted probabilities are between 0 and 1.

3. Log Link: This link function is used for count data and maps the linear predictor to the mean of a Poisson distribution. It ensures that the predicted mean is positive.

4. Inverse Link: This link function is used for modeling the mean of a Gamma distribution and maps the linear predictor to the inverse of the mean.

The choice of the link function depends on the nature of the response variable and the assumptions of the distribution. The link function allows for flexible modeling of the relationship between the linear predictor and the response variable, enabling the estimation of regression coefficients and making predictions within the appropriate range of the response distribution.

### Checking our inferences

Now is the time to check if our model's results are credible, via posterior predictive checks, to which we were introduced in the previous section.

In [ ]:
def plot_sneeze_predictions(idata, color="C0"):
    fig, axs = plt.subplots(2, 2, figsize=(12, 8))

    az.plot_ppc(
        idata,
        ax=axs[0, 0],
        coords={
            "obs_idx": np.where(np.logical_and(sneezes.alcohol == 0, sneezes.meds == 0))
        },
        color=color,
    )
    az.plot_ppc(
        idata,
        ax=axs[0, 1],
        coords={
            "obs_idx": np.where(np.logical_and(sneezes.alcohol == 0, sneezes.meds == 1))
        },
        color=color,
    )
    az.plot_ppc(
        idata,
        ax=axs[1, 0],
        coords={
            "obs_idx": np.where(np.logical_and(sneezes.alcohol == 1, sneezes.meds == 0))
        },
        color=color,
    )
    az.plot_ppc(
        idata,
        ax=axs[1, 1],
        coords={
            "obs_idx": np.where(np.logical_and(sneezes.alcohol == 1, sneezes.meds == 1))
        },
        color=color,
    )
    axs[0, 0].set_title("No alcohol : No meds")
    axs[0, 1].set_title("No alcohol : Meds")
    axs[1, 0].set_title("Alcohol : No meds")
    axs[1, 1].set_title("Alcohol : Meds")
    return fig, axs

In [ ]:
with m_sneeze:

    # Get posterior predictive samples, and add them to the InferenceData object
    trace_sneeze.extend(pm.sample_posterior_predictive(trace_sneeze))

In [ ]:
plot_sneeze_predictions(trace_sneeze);

While the model is in the ballpark of the actual data, it is not perfect. The model is underestimating the variance in the data. This is a common issue with Poisson regression, as the variance is constrained to be equal to the mean. When the data's mean and variance are not similar, a Poisson regression will underestimate the variance compared to the true variance observed in the data.

This behavior is quite common with Poisson regression: it often underestimates the variation in the data, simply because real data are more dispersed than our regression expects -- in these cases, data are said to be "overdispersed".

This phenomenon is particularly accute with the Poisson, because as we have seen its variance is mathematically constrained to be equal to its mean. So, when the data's mean and variance are not similar, a Poisson regression will get the variance estimate wrong when compared to the true variance observed in the data.

To convince ourselves, let's compare our data's mean and variance:

In [ ]:
sneezes.groupby(["meds", "alcohol"])["nsneeze"].agg(["mean", "var"])

Notice that for each combination of `alcohol` and `meds`, the variance of `nsneeze` is higher than the mean!

## Gamma-Poisson Model

Gamma-Poisson (aka [Negative binomial](https://en.wikipedia.org/wiki/Negative_binomial_distribution)) regression is used to model overdispersion in count data. The Gamma-Poisson distribution can be thought of as a Poisson distribution whose rate parameter is gamma distributed, so that rate parameter can be adjusted to account for the increased variance. If you want more details about these models (a.k.a. "continuous mixture models"), I refer you to chapter 12 of [Richard McElreath's excellent _Statistical Rethinking_](https://nbviewer.jupyter.org/github/pymc-devs/resources/blob/master/Rethinking_2/Chp_12.ipynb).

In addition to the Poisson rate, $\lambda$, Gamma-Poisson distributions are parametrized an additional overdispersion parameter, $\alpha$, which controls the shape of the Gamma distribution. 

_The Gamma-Poisson distribution_

We start with a random variable $Y$ that follows a Poisson distribution with rate $\lambda$. Turns out $\lambda$ is also random, and it follows a gamma distribution with parameters $\mu$ and $\alpha$

$$
\begin{aligned}
Y &\sim \text{Poisson}(\lambda) \\
\lambda &\sim \text{Gamma}\left(\mu, \alpha\right)
\end{aligned}
$$


We can marginalize over $\lambda$


$$
\begin{aligned}
p(y \mid \mu, \alpha) &= \int_0^{\infty}{p(y \mid \lambda) p(\lambda \mid \mu, \alpha) d\lambda} \\
&= \binom{y + \alpha - 1}{y}{\left(\frac{\alpha}{\mu + \alpha}\right)}^\alpha {\left(\frac{\mu}{\mu + \alpha}\right)}^y
\end{aligned}
$$


The above describes the probability mass function of a Gamma-Poisson distribution, then we can say 

$$
Y \sim \text{GammaPoisson}(\mu, \alpha)
$$


### Why is it useful?

Well, it relieves us from the previous constraint of our Poisson Distribution, to fix the **mean** to the **variance**.

<br> </br>

<center>
  <img src="images/poisson_gamma_poisson_drake.png" style="width:500px"; />
</center>

The common name for this type of model is [negative binomial](https://en.wikipedia.org/wiki/Negative_binomial_distribution) regression.  The negative binomial distribution has two parameters: the mean $\mu$ and the overdispersion parameter $\alpha$. The mean $\mu$ is the rate parameter of the Poisson distribution, and the overdispersion parameter $\alpha$ controls the variance of the distribution.

We'll use the following model...


$$
\begin{aligned}
\beta_{\text{intercept}} &\sim \mathrm{Normal}(0, 5)\\
\beta_{\text{alcohol}} &\sim \mathrm{Normal}(0, 1)\\
\beta_{\text{meds}} &\sim \mathrm{Normal}(0, 1) \\
\alpha &\sim \mathrm{Exponential}(1) \\
\mu_i &= \exp(\beta_{\text{intercept}} + \beta_{\text{meds}} \text{meds}_i + \beta_{\text{alcohol}} \text{alcohol}_i) \\
Y \mid \mu_i, \alpha &\sim \mathrm{NegativeBinomial}(\mu_i, \alpha) \\
\end{aligned}
$$

Let's use this likelihood in PyMC:

In [ ]:
with pm.Model(coords=COORDS) as m_sneeze_gp:
    # weakly informative priors
    a = pm.Normal("intercept", mu=0, sigma=5)
    b = pm.Normal("slopes", mu=0, sigma=1, dims="regressor")
    alpha = pm.Exponential("alpha", 1.0)

    # define linear model
    λ = pm.math.exp(a + b[0] * M + b[1] * A)

    ## likelihood
    y = pm.NegativeBinomial("y", mu=λ, alpha=alpha, observed=S, dims="obs_idx")

    trace_sneeze_gp = pm.sample()

In [ ]:
az.plot_trace(trace_sneeze_gp);

Sampling went well; let's quickly check the fit.

In [ ]:
with m_sneeze_gp:
    trace_sneeze_gp.extend(pm.sample_posterior_predictive(trace_sneeze_gp))

plot_sneeze_predictions(trace_sneeze_gp);

The predictions look much better than before, **but** we can see there is bias relativee to the observations in some groups. For example, in the plot for no alcohol with medication the model is biased to predict fewer sneezes than we actually observe in the medication condition, and the opposite phenomenon in the no medication condition. 

This suggests that we are missing some sort of **interaction effect** between medication and alcohol consumption. The thing is that our model is only able to account for the mean sneezes across both conditions.

Try your hand at adding an interaction term to the model:

In [ ]:
# Write your answer here

In [ ]:
COORDS["regressor"] = ["meds", "alcohol", "interaction"]
with pm.Model(coords=COORDS) as m_sneeze_inter:
    # weakly informative priors
    a = pm.Normal("intercept", mu=0, sigma=5)
    b = pm.Normal("slopes", mu=0, sigma=1, dims="regressor")
    α = pm.Exponential("α", 1.0)

    # linear model
    λ = pm.math.exp(a + b[0] * M + b[1] * A + b[2] * M * A)

    ## likelihood
    y = pm.NegativeBinomial("y", mu=λ, alpha=α, observed=S, dims="obs_idx")

    trace_sneeze_inter = pm.sample()

In [ ]:
az.plot_trace(trace_sneeze_inter);

We see that the slope for the interaction is reliably negative, meaning that taking meds when drinking alcohol will still tame the effects of the latter on sneezing and thus decrease the number of sneezes compared to not taking meds.

In [ ]:
with m_sneeze_inter:
    trace_sneeze_inter.extend(pm.sample_posterior_predictive(trace_sneeze_inter))

plot_sneeze_predictions(trace_sneeze_inter);

We can see that the interaction term appears to have removed the biases, and the predictions look much better across the board.

## Generalized Linear Models 

Poisson and negative binomial regressions are particular types of **Generalized Linear Model**. 

### Linear Models

We assume the conditional distribution of the response variable is a normal distribution. We model the mean of that normal distribution with a linear combination of the predictors. Mathematically, we have

$$
\begin{aligned}
\pmb{\beta} &\sim \mathcal{P}_{\pmb{\beta}} \\
\sigma &\sim \mathcal{P}_\sigma \\
\mu_i &= \beta_0 + \beta_1 X_{1i} + \cdots + \beta_p X_{pi} \\
Y_i \mid \mu_i, \sigma &\sim \text{Normal}(\mu_i, \sigma)
\end{aligned}
$$

where $\mathcal{P}_{\pmb{\beta}}$ is the joint prior for the regression coefficients and $\mathcal{P}_\sigma$ is the prior on the residual standard deviation.

### Generalized Linear Models

In Generalized Linear Models, we are not restricted to normal likelihoods and we model a function of the mean with a linear combination of the predictors.

$$
\begin{aligned}
\pmb{\beta} &\sim \mathcal{P}_{\pmb{\beta}} \\
\pmb{\theta} &\sim \mathcal{P}_{\pmb{\theta}} \\
g(\mu_i) &= \eta_i = \beta_0 + \beta_1 X_{1i} + \cdots + \beta_p X_{pi} \\
Y_i \mid \mu_i, \pmb{\theta} &\sim \mathcal{D}(\mu_i, \pmb{\theta})
\end{aligned}
$$

Which consists of:

* $\eta_i = \beta_0 + \beta_1 X_{1i} + \cdots + \beta_p X_{pi}$ is the **linear predictor**
* $g$ is the **link function**
    * In the Poisson regression model $g$ is the $\log$ function.
    * This point raises a lot of questions, as we directly work with the inverse link function $g^{-1}$ ($\exp$ in the previous case)
    * $g: \Omega \to \mathbb{R}$
    * $g^{-1}: \mathbb{R} \to \Omega$
    * $\Omega$ is the space of the mean parameter
* $\mathcal{D}$ is the **sampling distribution**
    * The conditional distribution of the response variable $Y$
    * Is not necessarily normal

Linear models are specific case of generalized linear models where $\mathcal{D} \equiv \mathcal{N}$ and $g = I$ (*i.e.* the identity function).

## Binomial regression

<img src="https://media.giphy.com/media/C9b7PXxqueVxK/giphy.gif" style="margin:auto" width="450"/>

In France, presidents are elected for five years. In the intervening time, polls are conducted to try and gauge the president's popularity, which often drives his re-election chances. The resulting *survey data* is complicated: measurements can be biased, the polling houses' sampling methods can be poor, response rates can be very low, etc. This provides many challenges for statisticians.

Generally, the polls are trying to answer one question: if a referendum were held today, who would win the presidential election? In other words, polls are a noisy estimate of the president's true, latent popularity. 

### French Polling Data

Let's import some polling data for Emmanuel Macron, the current French president.

In [ ]:
polls = pd.read_csv("../data/macron_popularity.csv")
polls

The most important column here is `N_approve`, the number of people who approve of Macron. What we want is a model that takes these polls as raw inputs and infers the true **latent proportion** of people who do approve of the president.

### The Model

The polls can be thought of as realizations of a Binomial sampling process: for each poll, a number $N_{\text{total}}$ of people are surveyed, and $N_{\text{approve}}$ of them say they approve of the president's job. Statistically speaking, we have 


$$N_{\text{approve}} \sim \mathrm{Binomial}(N_{\text{total}}, p_{\text{approve}})$$


where $p_{\text{approve}}$ equals the proportion of people supporting the president. So the simplest Bayesian model involves simply assigning a prior distribution to $p_{\text{approve}}$.

A natural prior for $p_{\text{approve}}$ is the beta distribution, which is the conjugate prior for the Binomial distribution. 

Beta distributions are parametrized by two positive reals, $\alpha$ and $\beta$, so called *shape* paramters. It can be difficult to gain an intuition about various combinations of $\alpha$ and $\beta$, so we can adopt an alternative paramterization of the beta distribution, $\mu$ and $\sigma$, the mean and standard deviation paramterization. Thus, we can more easily specify a prior based on how much *prior data* we base our beliefs on.

French presidents are usually not that popular, at least compared to the US. So a mean approval around 40% seems reasonable; For $\sigma$, we _know_ presidential approval never goes below 10% or above 90%. So, something like $\sigma = 0.15$ seems to fit the bill. You can play around with the code below to get a sense of how the Beta family behaves:

In [ ]:
mu, sigma = 0.4,0.15

ax = az.plot_dist(
    pm.draw(pm.Beta.dist(mu=mu, sigma=sigma), draws=10_000)
)
ax.set(
    title=f"Random draws from $Beta(\\mu={mu}, \\sigma={sigma})$",
    xlabel="Prior Popularity",
    ylabel="Plausibility",
);

The mathematical link between $(\mu, \sigma)$ and $(\alpha, \beta)$ is as follows:

$$
\begin{aligned}
\kappa &= \frac{\mu * (1 - \mu)}{\sigma^2} - 1 \\
\alpha &= \mu \times \kappa \\
\beta &= (1 - \mu) \times \kappa 
\end{aligned}
$$

which for our model translates to $\alpha = 3.9$ and $\sigma = 5.8$.

Our model is then:

$$
\begin{aligned}
p_{\text{approve}} &\sim \mathrm{Beta(\mu=0.4, \sigma=0.15)}\\
N_{\text{approve}} &\sim \mathrm{Binomial}(N_{\text{total}}, p_{\text{approve}})\\
\end{aligned}
$$


Let's code that up and run it in PyMC:

In [ ]:
with pm.Model() as m_raw:
    p_approve = pm.Beta("p_approve", mu=0.4, sigma=0.15)
    n_approve = pm.Binomial(
        "n_approve",
        n=polls.N_total.to_numpy(),
        p=p_approve,
        observed=polls.N_approve.to_numpy(),
    )

    trace_raw = pm.sample()
az.summary(trace_raw, round_to=3)

In [ ]:
az.plot_trace(trace_raw);

The posterior distribution seems confident (conditional on this model) that the president's approval rating is in the rang of 38.1% to 38.4%, which is rather narrow!

It's because we have more than 300 polls, which is a lot. 

Of course, not all of the polling data should be treated equally: those from 2017 should probably be discarded or down-weighted when trying to estimate 2021 popularity. Also, simple Binomial regressions tend to give overconfident estimates, for reasons similar to the Poisson, that we outlined above. If you add to that the fact that polls are noisy observations, which we did not account for in the model, there are reasons to doubt our model's confidence.

Actually, let's visualize posterior predictions of our model and see whether our skepticism is warranted:

In [ ]:
with m_raw:
    trace_raw.extend(pm.sample_posterior_predictive(trace_raw))
az.plot_ppc(trace_raw);

This is somewhat confusing because the default visualization returned by ArviZ is using different bins for the different posterior draws, the posterior predictive mean, and the observed data. Before making any conclusions about the model let's customize the visualization a bit. 

In [ ]:
def adjust_lightness(color, amount=0.5):
    import colorsys

    import matplotlib.colors as mc

    try:
        c = mc.cnames[color]
    except:
        c = color
    c = colorsys.rgb_to_hls(*mc.to_rgb(c))
    return colorsys.hls_to_rgb(c[0], c[1] * amount, c[2])


# Generate some graded colors
colors = [adjust_lightness("C0", x) for x in [1.8, 1.6, 1.4, 1.2, 0.9]]

# Quantiles corresponding to color gradings (used in plot below)
quantile_bands = [(0.05, 0.95), (0.15, 0.85), (0.25, 0.75), (0.35, 0.65)]


def plot_ppc_discrete(idata, bins, ax):
    # Add uncertainty bands around a line in plot
    def add_discrete_bands(x, lower, upper, ax, **kwargs):
        for i, (l, u) in enumerate(zip(lower, upper)):
            s = slice(i, i + 2)
            ax.fill_between(x[s], [l, l], [u, u], **kwargs)

    # Get data variable name
    var_name = list(idata.observed_data.data_vars)[0]
    # Extract observations form idata
    y_obs = idata.observed_data[var_name].to_numpy()

    counts_list = []
    for draw_values in az.extract(idata, "posterior_predictive")[var_name].to_numpy().T:
        counts, _ = np.histogram(draw_values, bins=bins)
        counts_list.append(counts)
    counts_arr = np.stack(counts_list)

    # Add quantile bands and median to plot
    for i in range(len(quantile_bands)):
        qts_tmp = np.quantile(counts_arr, quantile_bands[i], axis=0)
        add_discrete_bands(bins, qts_tmp[0], qts_tmp[1], ax=ax, color=colors[i])

    median = np.quantile(counts_arr, 0.5, axis=0)

    ax.step(bins[:-1], median, color=colors[4], lw=2, where="post")

    # Add ground truth to plot
    ax.hist(y_obs, bins=bins, histtype="step", lw=2, color="black", align="mid")

    # Add legend
    handles = [
        Line2D([], [], label="Observed data", color="black", lw=2),
        Line2D([], [], label="Posterior predictive median", color=colors[4], lw=2),
    ]
    ax.legend(handles=handles)

    return ax

In [ ]:
fig, ax = plt.subplots()
_, bins = np.histogram(polls["N_approve"], bins=50)
plot_ppc_discrete(trace_raw, bins, ax)
ax.set(xlabel="Number of approvals");

So, the model is clearly not a great fit to the data. It's not _that_ surprising, since this model is very simple, while approval ratings are the results of complex socio-economic interactions that evolve with time.

### Adding a predictor

A first thing we can do is help our model by adding a variable that is thought to be correlated with presidential approval. For example, the unemployment rate, which we actually already have in our dataset.

More precisely, we will use the _logarithm_ of unemployment, not the raw unemployment rate, which is easier to work with.

$$
\begin{aligned}
\text{baseline} &\sim \mathrm{Normal}(-0.7, 0.5)\\
\beta_{\text{unemp}} &\sim \mathrm{Normal}(0, 0.2)\\
p_{\text{approve}} &= \text{logit}^{-1}(\text{baseline} + \beta_{\text{unemp}} \times \log(\text{unemp}))\\
N_{\text{approve}} &\sim \mathrm{Binomial}(N_{\text{total}}, p_{\text{approve}})\\
\end{aligned}
$$

Note that here we are using a **logit link** function. This has the same role as the exponential in the Poisson regression, except that the logit function links the real line (here, the parameter space, where $baseline$ and $\beta_{\text{unemp}}$ live) to the unit interval line (here, the outcome space, where $p_{\text{approve}}$ lives).

$$x = \text{logit}(p) = \log\left[\frac{p}{1-p}\right]$$

$$p = \text{logit}(x)^{-1} = \frac{1}{1 + e^{-x}}$$

Let's visualize this function to get a better understanding:

In [ ]:
def invlogit(x):
    return 1 / (1 + np.exp(-x))

In [ ]:
xvals = np.linspace(-8, 8)
plt.plot(xvals, invlogit(xvals))
plt.xlabel("parameter space")
plt.ylabel("probability space");

To gain an intuition, notice that 0 on the logit space translates to a probability of 50%. Similarly, anything below about -5 is close to 0%, and anything above 5 is close to 100%.

This means that the logistic strongly distorts the parameter space: only the numbers on $]-6, 6[$ have a probability different than 0 or 1. So priors that you _think_ are reasonable on the parameter space might turn out to be completely unreasonable on the probability space. So, with GLMs you have to be careful when choosing priors, because of the distortion of space caused by the link function.

#### Selecting priors

Our regression intercept is the baseline popularity. Suppose we behaved like this was an ordinary least squares regression and chose something like a $\mathrm{Normal}(0, 10)$ prior, which usually works well for the intercept of a linear regression. What does this mean on the outcome space?

In [ ]:
from scipy.special import expit as logistic

ax = az.plot_kde(
    logistic(pm.draw(pm.Normal.dist(mu=0, sigma=10), draws=20_000)),
    label="baseline ~ $Normal(0, 10)$",
)
ax.set_xlim((0, 1))
ax.set_xlabel("Baseline presidential popularity")
ax.set_ylabel("Density")
ax.set_title("Baseline prior");

This encodes exactly the opposite of our domain knowledge! 

We can do better: we expect most presidents to have a baseline popularity between 20% and 50% -- in other words, French people rarely love their presidents but often _really_ dislike them. $\mathrm{Normal}(-0.7, 0.5)$ looks reasonable in that regard: it expects 95% of the probability mass to be between -1.7 and 0.3, i.e $logistic(-1.7) = 15\%$ and $logistic(0.3) = 57\%$, with a mean approval of $logistic(-0.7) = 33\%$:


> Remember our lecture on choosing priors. If you want to e.g. have $95\%$ of the mass in the prior on $\text{logit}^{-1}(p_{approve})$ lie within $15\%$ and $57\%$, you could also just check for the respective inputs to the `invlogit` function ($-1.7$ and $0.3$ from above) and then request prior parameters via the,`pm.find_constrained_prior` function!

In [ ]:
baseline_prior_samples = pm.draw(pm.Normal.dist(-0.7, 0.5), draws=20_000)

ax = az.plot_kde(
    invlogit(baseline_prior_samples),
    label="baseline ~ $Normal(-0.7, 0.5)$",
)
ax.set_xlim((0, 1))
ax.set_xlabel("Baseline presidential popularity")
ax.set_ylabel("Density")
ax.set_title("Baseline prior");

Much better!

What about the effect of unemployment rate? Some domain knowledge suggests we should expect a mild effect. Unemployment is one of several factors influencing voters' opinion of the president, and partisanship makes movements in popularity less responsive to unemployment -- if you really don't like the president, you probably need to see a very low unemployment rate before starting to give them credit.

All in all, we expect the unemployment to have a small negative effect, but its difficult to quantify. So, let's center our prior on $0$ (i.e no expected effect) and use a weakly regularizing $\sigma$ (in log-odds space): $\beta_{\text{unemp}} \sim \mathrm{Normal}(0, 0.2)$. To see the effect of this prior, we have to plug it into the formula for our model

$$p_{\text{approve}} = \text{logit}^{-1}(\text{baseline} + \beta_{\text{unemp}} \times \log(\text{unemp}))$$

It is useful to simulate some unemployment data. For convenience, we will standardize the real unemployment data (*i.e.* force it to have mean 0 and standard deviation 1), which makes it easier to set our priors, and as a further benefit, the sampler may have an easier time converging.

In [ ]:
def standardize(series):
    """Standardize a pandas series"""
    return (series - series.mean()) / series.std()


stdz_log_unemployment = standardize(np.log(polls.unemployment)).to_numpy()

So, as the data are standardized, simulating a grid of unemployment between -3 and 3 is largely sufficient to cover the whole range of possible data:

In [ ]:
unemp_effect_prior_samples = pm.draw(pm.Normal.dist(0.0, 0.2), draws=20_000)
unemp_grid = np.linspace(-3, 3, 200)

prior_approval = invlogit(
    baseline_prior_samples[:, None] + unemp_effect_prior_samples[:, None] * unemp_grid
)

for i in range(100):
    plt.plot(unemp_grid, prior_approval[i], "k", alpha=0.2)
plt.xlabel("$log(unemp)$ (standardized)")
plt.ylabel("prior approval")
plt.title("$\\text{baseline} ~ Normal(-0.7, 0.5), \\beta_{\\text{unemp}} \\sim Normal(0, 0.2)$");

Each line is a possible relationship, according to our model's assumptions, between unemployment and latent popularity, factoring in the baseline effect. Each line shows a smooth relationship between unemployment and popularity. 

We could add a constraint that popularity only decreases with increasing unemployment, but we will not do that here.

#### Exercise

Implement the covariate model in PyMC and sample from it. Assign the output from `pm.sample` to an object called `trace_unemp`. 

In [ ]:
# Write your answer here

In [ ]:
az.plot_trace(trace_unemp);

Let's check out the posterior predictive distribution of approvals, and compare it to the naive model:

In [ ]:
with m_unemp:
    trace_unemp.extend(pm.sample_posterior_predictive(trace_unemp))

In [ ]:
_, axes = plt.subplots(1, 2, figsize=(12, 5), constrained_layout=True, sharey=True)
_, bins = np.histogram(polls["N_approve"], bins=50)

plot_ppc_discrete(trace_raw, bins, axes[0])
plot_ppc_discrete(trace_unemp, bins, axes[1])

axes[0].set(title="Model 0: No predictors", xlabel="Number of approvals")
axes[1].set(title="Model 1: log(unemployment)", xlabel="Number of approvals");

The model shows modest improvement in the overall shape of the predictions, but it still has trouble with low approval ratings in particular. It is also quite confident in its predictions, which is not ideal.


## Beta-Binomial regression

As noted, binomial regressions can give overconfident estimates, for reasons similar to the Poisson: the binomial distribution only has one parameter, which closely ties the variance to the expected value:

$$\text{E}(Y) = N_{\text{total}} \times p_{\text{approve}}$$
$$\text{Var}(Y) = N_{\text{total}} \times p_{\text{approve}} \times (1 - p_{\text{approve}})$$

So, as we used the Gamma-Poisson distribution to give more flexibility to the Poisson, we can similarly allow the binommial probability to vary according to some distribution as well. Canonically, we use the [Beta-Binomial distribution](https://en.wikipedia.org/wiki/Beta-binomial_distribution), which handles data that is overdispersed relative to the binomial distribution.

Under this model, the binomial probabilities are no longer fixed, but are rather random variables drawn from a common beta distribution. So, in addition to the number of trials, $n$, Beta-Binomial distributions are parametrized by the beta distribution's shape parameters, $\alpha$ and $\beta$. 

We will specify a covariate model for $\mu$ and a place a prior on $\kappa$ rather than on $\sigma$, since $\sigma$ is awkwardly constrained to the interval $[0, \sqrt(\mu (1 - \mu))]$. In the previous model, the value of $\kappa$ was approximately 10.

Here, we will specify:

$$
\begin{aligned}
\text{baseline} &\sim \mathrm{Normal}(-0.7, 0.5)\\
\beta_{\text{unemp}} &\sim \mathrm{Normal}(0, 0.2)\\
p_{\text{approve}} &= \text{logit}^{-1}(\text{baseline} + \beta_{\text{unemp}} \times \log(\text{unemp}))\\
\kappa &\sim \mathrm{Exponential}(1) + 10\\
N_{\text{approve}} &\sim \mathrm{BetaBinomial}(\alpha=p_{\text{approve}} \times \kappa, \beta = (1 - p_{\text{approve}}) \times \kappa, \: N_{\text{total}})
\end{aligned}
$$

So we are assuming that $\kappa$ is *at least* 10, allowing the variance to be larger if necessary. To that end, we can use a trick and define $\kappa = \tilde{\kappa} + 10$, where $\tilde{\kappa} \sim \mathrm{Exponential}(1)$, which works because exponential distributions have a minimum of zero. 

This translates easily into PyMC:

In [ ]:
U = stdz_log_unemployment
N, Y = polls[['N_total', 'N_approve']].to_numpy().T

with pm.Model() as m_betabin:

    # intercept on logit scale
    baseline = pm.Normal("baseline", -0.7, 0.5)
    # log unemployment slope
    log_unemp_effect = pm.Normal("log_unemp_effect", 0.0, 0.2)

    # invlogit --> logistic
    p_approve = pm.Deterministic("p_approve", pm.math.invlogit(baseline + log_unemp_effect * U))

    # overdispersion parameter
    kappa = pm.Exponential("kappa_offset", 1.0) + 10.0

    n_approve = pm.BetaBinomial(
        "n_approve",
        alpha=p_approve * kappa,
        beta=(1.0 - p_approve) * kappa,
        n=N,
        observed=Y,
    )

    trace_betabin = pm.sample()

az.summary(trace_betabin, var_names="~p_approve", round_to=2)

In [ ]:
az.plot_trace(trace_betabin, var_names="~p_approve");

Note how the posterior distributions of `baseline` and `log_unemp_effect` are much wider than before -- that's already a good sign.

Let's see how this propagates to the posterior predictions:

In [ ]:
with m_betabin:
    trace_betabin.extend(pm.sample_posterior_predictive(trace_betabin))

In [ ]:
_, axes = plt.subplots(1, 3, figsize=(16, 6), constrained_layout=True, sharey=True)
_, bins = np.histogram(polls["N_approve"], bins=50)

plot_ppc_discrete(trace_raw, bins, axes[0])
plot_ppc_discrete(trace_unemp, bins, axes[1])
plot_ppc_discrete(trace_betabin, bins, axes[2])

axes[0].set(title="Model 0: No predictors", xlabel="Number of approvals")
axes[1].set(title="Model 1: log(unemployment)", xlabel="Number of approvals")
axes[2].set(
    title="Model 2: log(unemployment) + BetaBinomial", xlabel="Number of approvals"
);

That's the best we've seen so far! 

This is still not a production-grade model, but considering how simple it sill is it does not perform badly. Polls really are notoriously noisy, and the beta-binomial seems to characterize this noise pretty well.

Our predictions still aren't very useful though; while they are a reasonable data-generating model, its because we have cranked up the model's variance. 

### How to improve this model?

We could add more structure to the model to make our predictions more precise. For example, we could think about adding relevant predictor variables, using polling data from previous presidents (approval works in cycle, no matter who's president, so knowing about previous presidents would help), and adding a time component (we already noticed that polls from 2017 shouldn't weigh as much as polls from 2021). This last option would also allow us to do time series predictions, which is intrinsically interesting here.


### Evaluating the unemployment effect

Clearly, the biggest improvement in our model was the result of using a beta-binommial likelihood in place of the simpler binomial. So, its not clear how much the unemployment rate actually helped.

One way to evaluate this is to look at the implications of varying unemployment values on the model predictions. We can do this without fitting a new model, using the samples from the posterior distribution in the trace object.

In [ ]:
trace_betabin.posterior

Let's use our grid of unemployment values as a *counterfactual* and see how the model's predictions change as we move from low to high unemployment rates, had they been observed in the data.

In [ ]:
unemp_grid = xr.DataArray(unemp_grid, dims="counterfactual")

We push these values through the model, using the values sampled for the model parameters in the trace.

In [ ]:
post_approval = invlogit(trace_betabin.posterior["baseline"] + trace_betabin.posterior["log_unemp_effect"] * unemp_grid)

In [ ]:
_, ax = plt.subplots(1, 1, figsize=(12, 5))

az.plot_hdi(unemp_grid, post_approval, ax=ax, backend_kwargs={"label": "Posterior HDI"})
ax.plot(unemp_grid, post_approval.mean(("chain", "draw")), ls="--", lw="3", label="Posterior mean")
ax.scatter(stdz_log_unemployment, polls["N_approve"] / polls["N_total"], c="k", alpha=0.1, label="Observed")

ax.set(xlabel="$log(unemp)$ (standardized)", ylabel="Mean approval", title="Conditional Adjusted Predictions")
ax.legend();

The effect of unemployment is positive with respect to approval rating! This seems like a suspicious result.

### Exercise

Code the model without the unemployment data, but with the Beta-Binomial noise distribution. Assign the output from `pm.sample` to an object called `trace_no_unemp`.

In [ ]:
# Write your answer here

In [ ]:
az.summary(trace_betabin_no_unemp, round_to=2)

In [ ]:
az.plot_trace(trace_betabin_no_unemp);

In [ ]:
with m_betabin_no_unemp:
    trace_betabin_no_unemp.extend(
        pm.sample_posterior_predictive(trace_betabin_no_unemp)
    )

In [ ]:
_, axes = plt.subplots(2, 2, figsize=(16, 6), constrained_layout=True, sharey=True)
_, bins = np.histogram(polls["N_approve"], bins=50)

plot_ppc_discrete(trace_raw, bins, axes[0, 0])
plot_ppc_discrete(trace_unemp, bins, axes[1, 0])
plot_ppc_discrete(trace_betabin, bins, axes[0, 1])
plot_ppc_discrete(trace_betabin_no_unemp, bins, axes[1, 1])


axes[0, 0].set(title="Model 0: No predictors", xlabel="Number of approvals")
axes[1, 0].set(title="Model 1: log(unemployment)", xlabel="Number of approvals")
axes[0, 1].set(
    title="Model 2: log(unemployment) + BetaBinomial", xlabel="Number of approvals"
)
axes[1, 1].set(title="Model 3: BetaBinomial", xlabel="Number of approvals");

Another look at the data shows that something else must be going on ...

In [ ]:
time = pd.to_datetime(polls["date"])

_, (up, down) = plt.subplots(2, 1, figsize=(12, 6), sharex=True)

up.scatter(time, stdz_log_unemployment, c="C1", alpha=0.5)
up.set(ylabel="Unemployment")

down.scatter(time, polls["N_approve"] / polls["N_total"], c="C0", alpha=0.5)
down.set(ylabel="Approval");

### Exercise: Titanic survival model

Load the titanic dataset, and construct an appropriate model to predict passenger survival rate. Summarize the parameter estimates.

In [ ]:
try:
    titanic = pd.read_excel('../data/titanic.xls')
except FileNotFoundError:
    titanic = pd.read_excel(DATA_URL + 'titanic.xls')
titanic.head()

In [ ]:
# Write your answer here

In [ ]:
%load_ext watermark
%watermark -n -u -v -iv -w